In [ ]:
### LIBRARY
import os
import glob
from skimage import io
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from skimage.io import imsave
from skimage.filters import threshold_otsu
from skimage.filters import try_all_threshold
from skimage.filters import threshold_minimum, threshold_triangle, threshold_yen, threshold_mean
from skimage.filters import threshold_li, threshold_isodata
from skimage.filters import gaussian
from skimage.morphology import label
from skimage.measure import regionprops
from skimage.morphology import remove_small_objects
from skimage.morphology import remove_small_holes
from skimage.morphology import dilation, disk
from skimage.morphology import closing
from skimage.exposure import equalize_adapthist
from natsort import natsorted

In [ ]:
### PARAMETERS
filename = ''
# filename extract
last_folder = os.path.basename(filename)
# output folder:
output_folder = ''


# to save masks:
folder = f'{filename}/WT1_masks'
os.makedirs(folder, exist_ok=True)

print(last_folder)

pixel_size = 0.65

In [ ]:
### LOADING

## DAPI in 405 channel:
list_of_dapi_files = natsorted(glob.glob(f"{filename}/*ch00*.tif"))
images_dapi = {}
for file in list_of_dapi_files:
    img = io.imread(file)
    images_dapi[os.path.basename(file)] = img

## 488 channel:
list_of_488_files = natsorted(glob.glob(f"{filename}/*ch01*.tif"))
images_488 = {}
for file in list_of_488_files:
    img = io.imread(file)
    images_488[os.path.basename(file)] = img

## 594 channel:
list_of_594_files = natsorted(glob.glob(f"{filename}/*ch02*.tif"))
images_594 = {}
for file in list_of_594_files:
    img = io.imread(file)
    images_594[os.path.basename(file)] = img

## 647 channel:
list_of_647_files = natsorted(glob.glob(f"{filename}/*ch03*.tif"))
images_647 = {}
for file in list_of_647_files:
    img = io.imread(file)
    images_647[os.path.basename(file)] = img

print(f"to check: \n number of 405 images: \t {len(images_dapi)}, \n number of 488 images: \t {len(images_488)},"
      f"\n number of 594 images: \t {len(images_594)}, \n number of 647 images: \t {len(images_647)}")

In [ ]:
# ### PARAMETERS testing

tested_image = '' #Image 1

## testing:
# tries all automatic thresholding options for the image 
# fig, ax = try_all_threshold(blurred_dapi_test, figsize=(6, 12), verbose=False)
# plt.show()

## DAPI/organoid area:
def dapi_analysis(image):
    dapi_blurring = gaussian(image, sigma=1)
    #dapi_image_clipped = np.clip(image, a_min=0, a_max=500) # use for burned pixels
    thresholding = dapi_blurring > threshold_otsu(dapi_blurring)
    thresholding_2 = remove_small_holes(thresholding, area_threshold=80000)
    dapi_blurring_2 = gaussian(thresholding_2, sigma=20)
    thresholding_3 = dapi_blurring_2 > threshold_triangle(dapi_blurring_2)
    thresholding_4 = remove_small_holes(thresholding_3, area_threshold=80000)
    thresholding_5 = remove_small_objects(thresholding_4, min_size=80000)
    return(thresholding_5)

## 488
def analysis_488(image):
    thresholding_1 = image > threshold_triangle(image)
    thresholding_2 = remove_small_objects(thresholding_1, min_size=200)
    return(thresholding_2)

## 594
def analysis_594(image):
    image_clipped = np.clip(image, a_min=400, a_max=5000) # use for burned pixels
    thresholding_1 = image_clipped > threshold_triangle(image_clipped)
    thresholding_2 = remove_small_objects(thresholding_1, min_size=50)
    mask_dilated = dilation(thresholding_2, disk(25))
    mask_closed = closing(mask_dilated, disk(10))
    thresholding_3 = remove_small_holes(mask_closed, area_threshold=10000)

    return(thresholding_3)

## 647
def analysis_647(image):
    normalisation = equalize_adapthist(image, clip_limit=0.004) #0.009
    thresholding_1 = normalisation > threshold_triangle(normalisation) #NOTE: using a specific threshold 
                                                                       #value is usually fastest for individual images
    thresholding_2 = remove_small_objects(thresholding_1, min_size=150)
    return(thresholding_2)


image_dapi_test = images_dapi[f'{tested_image}_ch00.tif']
image_thresholded = dapi_analysis(image_dapi_test)

fig, axs = plt.subplots(ncols=2, figsize=(10,7))
axs[0].imshow(image_dapi_test)
axs[1].imshow(image_thresholded)

image_488_test = images_488[f'{tested_image}_ch01.tif']
thresholded_488_test = analysis_488(image_488_test)


image_594_test = images_594[f'{tested_image}_ch02.tif']
thresholded_594_2_test = analysis_594(image_594_test)


image_647_test = images_647[f'{tested_image}_ch03.tif']
thresholded_647_2_test = analysis_647(image_647_test)


fig, axs = plt.subplots(ncols=3, figsize=(10,7))
axs[0].imshow(thresholded_488_test)
axs[1].imshow(thresholded_594_2_test)
axs[2].imshow(thresholded_647_2_test)


In [ ]:
### Analysis
Image_data_CD31_files = {}

for name,img in images_dapi.items():

    ## DAPI
    thresholded_405 = dapi_analysis(img)
    organoid_size = np.sum(thresholded_405)
    organoid_size_in_um = organoid_size*(pixel_size**2)

    #image name extracted:
    name_without_last_part = '_'.join(name.split('_')[:-1]) # splits the ch__ from the name, so that
                                                              # the script can get the relevant images 
                                                              # from other channels

    ## 488
    img_488 = images_488[f'{name_without_last_part}_ch01.tif']
    thresholded_488 = analysis_488(img_488) & thresholded_405
    overal_488_signal = np.sum(thresholded_488) # records overal 488 area

    if overal_488_signal == 0:
        overal_488_signal = np.nan

    ## 594
    img_594 = images_594[f'{name_without_last_part}_ch02.tif']
    threshold_594 = analysis_594(img_594) & thresholded_405
    overal_594_signal = np.sum(threshold_594) # records overal 488 area

    if overal_594_signal == 0:
        overal_594_signal = np.nan

    ## 647
    img_647 = images_647[f'{name_without_last_part}_ch03.tif']
    threshold_647 = analysis_647(img_647) & thresholded_405
    overal_647_signal = np.sum(threshold_647) # records overal 488 area
    labeled_647 = label(threshold_647)
    objects_647 = regionprops(labeled_647, img_647)

    if overal_647_signal == 0:
        overal_647_signal = np.nan

    ## calculations:
    WT1_area = overal_594_signal/organoid_size
    TWIST_area = overal_647_signal/organoid_size
    macs_area = overal_488_signal/organoid_size

    TWIST1_number = labeled_647.max()

    ## are GFP+ in WT1 areas?
    GFP_in_WT1 = np.sum(thresholded_488 & threshold_594)
    if GFP_in_WT1==0:
        GFP_in_WT1_per = np.nan
    else:
        GFP_in_WT1_per = GFP_in_WT1/overal_488_signal

    # GFP in the rest of the organoid:
    GFP_in_the_rest = np.sum(thresholded_488 & ~threshold_594)
    if GFP_in_the_rest==0:
        print(f'no macs in the rest of {name_without_last_part}')
        GFP_per_in_rest = np.nan
    else:
        GFP_per_in_rest = GFP_in_the_rest/overal_488_signal
    

    # image mask:
    R = (
        threshold_594.astype(np.uint8) * 255 + # Red from Magenta
        threshold_647.astype(np.uint8) * 255
    )
    G = (
        thresholded_488.astype(np.uint8) * 255 + # Green from Green
        threshold_647.astype(np.uint8) * 255
    )
    B = (
        thresholded_405.astype(np.uint8) * 255 +  # Blue from Blue
        threshold_594.astype(np.uint8) * 255   # Blue from Magenta
    )
    # Stack into RGB
    rgb_image = np.stack([R, G, B], axis=-1)
    # Clip values to stay within valid 8-bit range
    rgb_image = np.clip(rgb_image, 0, 255).astype(np.uint8)

    image_filename = f'{name_without_last_part}.png'
    filepath = os.path.join(folder, image_filename)
    plt.imsave(filepath, rgb_image)

    # image data
    Image_data_CD31_files[name_without_last_part] = {
        'organoid area': organoid_size,
        'GFP area': macs_area,
        'TWIST1 area': TWIST_area,
        'TWIST1 number': TWIST1_number,
        'macs in WT1 area': GFP_in_WT1_per,
        'macs in the rest per': GFP_per_in_rest,
    }

In [ ]:
### DATAFRAME
df_stain1 = pd.DataFrame.from_dict(Image_data_CD31_files, orient='index')

output_path = f'{output_folder}/{last_folder}_WT1.xlsx'
df_stain1.to_excel(output_path)
